## Bài 9: Hybrid Algorithms

### Cài đặt MA cho bài toán TSP

In [11]:
import random
import numpy as np


def distance_matrix(coords):
    n = len(coords)
    dmat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dmat[i, j] = np.linalg.norm(
                np.array(coords[i]) - np.array(coords[j]))
    return dmat


def tour_length(tour, dmat):
    return sum(dmat[tour[i], tour[(i+1) % len(tour)]] for i in range(len(tour)))


def init_population(pop_size, n_cities):
    return [random.sample(range(n_cities), n_cities) for _ in range(pop_size)]

def fitness(tour, dmat):
    return -tour_length(tour, dmat)


def tournament_selection(pop, dmat, k=3):
    candidates = random.sample(pop, k)
    return max(candidates, key=lambda t: fitness(t, dmat))


def order_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = p1[a:b]
    ptr = b
    for x in p2[b:] + p2[:b]:
        if x not in child:
            if ptr >= size:
                ptr = 0
            child[ptr] = x
            ptr += 1
    return child


def swap_mutation(tour):
    a, b = random.sample(range(len(tour)), 2)
    tour[a], tour[b] = tour[b], tour[a]
    return tour


def two_opt(tour, dmat):
    improved = True
    best = tour
    best_len = tour_length(best, dmat)
    while improved:
        improved = False
        for i in range(len(tour)-1):
            for j in range(i+2, len(tour)):
                if j-i == 1:
                    continue
                new_tour = best[:i] + best[i:j][::-1] + best[j:]
                new_len = tour_length(new_tour, dmat)
                if new_len < best_len:
                    best, best_len = new_tour, new_len
                    improved = True
    return best


def memetic_tsp(coords, pop_size=50, gens=100, p_local=0.3):
    dmat = distance_matrix(coords)
    pop = init_population(pop_size, len(coords))
    best = min(pop, key=lambda t: tour_length(t, dmat))
    for g in range(gens):
        new_pop = []
        while len(new_pop) < pop_size:
            p1 = tournament_selection(pop, dmat)
            p2 = tournament_selection(pop, dmat)
            c = order_crossover(p1, p2)
            c = swap_mutation(c)

            if random.random() < p_local:
                c = two_opt(c, dmat)

            new_pop.append(c)
        pop = new_pop
        gen_best = min(pop, key=lambda t: tour_length(t, dmat))
        if tour_length(gen_best, dmat) < tour_length(best, dmat):
            best = gen_best
        print(f"Gen {g}:")
        print(f"Best path = {tour_length(best, dmat):.3f}")
        print(f"Best length = {best}")

    return best, tour_length(best, dmat)

In [13]:
random.seed(42)
coords = [(random.random()*100, random.random()*100) for _ in range(30)]
best_tour, best_len = memetic_tsp(coords, pop_size=30, gens=50, p_local=0.3)
print("Best tour:", best_tour)
print("Best length:", best_len)

Gen 0:
Best path = 433.343
Best length = [5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0, 20, 9, 3, 12, 19, 14, 18, 10, 15, 2, 24, 7, 27, 16, 26, 28, 8, 17]
Gen 1:
Best path = 426.240
Best length = [20, 9, 3, 12, 24, 7, 27, 19, 14, 18, 10, 15, 2, 16, 26, 28, 8, 17, 5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0]
Gen 2:
Best path = 426.240
Best length = [20, 9, 3, 12, 24, 7, 27, 19, 14, 18, 10, 15, 2, 16, 26, 28, 8, 17, 5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0]
Gen 3:
Best path = 426.240
Best length = [20, 9, 3, 12, 24, 7, 27, 19, 14, 18, 10, 15, 2, 16, 26, 28, 8, 17, 5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0]
Gen 4:
Best path = 425.885
Best length = [3, 12, 19, 14, 18, 10, 15, 2, 27, 7, 24, 17, 16, 26, 28, 8, 5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0, 20, 9]
Gen 5:
Best path = 425.885
Best length = [3, 12, 19, 14, 18, 10, 15, 2, 27, 7, 24, 17, 16, 26, 28, 8, 5, 29, 23, 22, 6, 13, 21, 1, 25, 11, 4, 0, 20, 9]
Gen 6:
Best path = 425.885
Best length = [3, 12, 19, 14, 18, 10, 15, 2, 27, 7, 24, 17, 1